# Extended figure 6e

In [ ]:
import sys
import subprocess
%matplotlib inline
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import glob
import os
import harmonypy as hm
from pathlib import Path

#sys.path.append("/home/jovyan/Whole_Embryo/Whole_Embryo_useful_definitions/")
#import UsefulDefs as ud

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=300, dpi_save=300, color_map='viridis')

In [ ]:
adata = sc.read('/home/jovyan/YS_project/YS/Data_objects/final_objects/A4_V7_YS_integrated_data_singlets_with_raw_counts_for_MS_plotting_20211111_with_obsp.h5ad')

dict1 = {
'CS10' : 'CS10-11',
'CS11' : 'CS10-11',
'CS14' : 'CS14-15',
'CS15' : 'CS14-15',
'CS18' : 'CS18',
'CS22' : 'CS22-23',
'CS23' : 'CS22-23'
}

adata.obs['time_points'] = adata.obs['stage'].map(dict1)

# macrophage, erythroid, endoderm
mac_csv = pd.read_csv('/home/jovyan/YS_project/YS/YS_check_ealy_vs_late_csvs/sig_FC_early_late_neighbourhoods/Macrophage_sig_fc_early_late.csv', index_col = 1)
eryth_csv = pd.read_csv('/home/jovyan/YS_project/YS/YS_check_ealy_vs_late_csvs/sig_FC_early_late_neighbourhoods/Erythroid_sig_fc_early_late.csv', index_col = 1)
endo_csv = pd.read_csv('/home/jovyan/YS_project/YS/YS_check_ealy_vs_late_csvs/sig_FC_early_late_neighbourhoods/Endoderm_sig_fc_early_late.csv', index_col = 1)

#mac_csv = mac_csv[~mac_csv['nhood_Sig_FC'].str.match('False')]
#eryth_csv = eryth_csv[~eryth_csv['nhood_Sig_FC'].str.match('False')]
#endo_csv = endo_csv[~endo_csv['nhood_Sig_FC'].str.match('False')]

# Fix mismatching barcodes

In [ ]:
adata_index = list(adata.obs.index)

n = 0
data_names = ['mac_csv','eryth_csv','endo_csv']

for i in [mac_csv,eryth_csv,endo_csv]:
    index_list = i.index[i.index.str.contains('Wang')]
    diff = list(set(index_list).difference(adata_index))
    for index in diff:
        i = i.rename(index={index: index+'_1'})
    index_list = i.index[i.index.str.contains('Wang')]
    intersect = (set(adata_index) & set(index_list))
    if len(intersect) == len(index_list):
        name = data_names[n]
        globals()[name] = i
        n +=1
        pass
    else:
        diff = list(set(index_list).difference(adata_index))
        for index in diff:
            i = i.rename(index={index: index+'_2'})
        index_list = i.index[i.index.str.contains('Wang')]
        intersect = (set(adata_index) & set(index_list))
        if len(intersect) == len(index_list):
            globals()[name] = i
            n +=1
            pass
        else:
            raise Exception(f'Error in barcode inetersect on loop {n}!')

In [ ]:
macrophage = adata[adata.obs.index.isin(list(mac_csv.index))][:]
erythroid = adata[adata.obs.index.isin(list(eryth_csv.index))][:]
endoderm = adata[adata.obs.index.isin(list(endo_csv.index))][:]

macrophage.obs['nhood_Sig_FC'] = mac_csv['nhood_Sig_FC'][mac_csv.index.isin(list(macrophage.obs.index))]
erythroid.obs['nhood_Sig_FC'] = eryth_csv['nhood_Sig_FC'][eryth_csv.index.isin(list(erythroid.obs.index))]
endoderm.obs['nhood_Sig_FC'] = endo_csv['nhood_Sig_FC'][endo_csv.index.isin(list(endoderm.obs.index))]

In [ ]:
objects = [macrophage,erythroid,endoderm]

for i in objects:
    i.obs['early_late_time_points'] = i.obs['nhood_Sig_FC'].astype(str) + '_' + i.obs['time_points'].astype(str)

In [ ]:
endoderm = endoderm[~endoderm.obs['early_late_time_points'].isin(['Late_CS14-15']),:]

In [ ]:
datas = [macrophage,endoderm]
names = ['Macrophage','Endoderm']
orig_name = ['macrophage','endoderm']
n = 0

for i in datas:
    name = names[n]
    temp_data = adata[adata.obs['broad_cell_labels_organ'].isin([name+'_ys'])]
    temp_data = temp_data[temp_data.obs['stage'].isin(['CS22','CS23'])]
    datas = [i,temp_data]
    new_data = sc.AnnData.concatenate(*datas, join='inner', batch_categories=None ,index_unique=None)
    del new_data.obs['batch']
    old_name = orig_name[n]
    globals()[old_name] = new_data
    n+=1

macrophage.obs.loc[macrophage.obs['time_points'].isin(['CS22-23']),['nhood_Sig_FC','early_late_time_points']] = ['Late','Late_CS22-23']
endoderm.obs.loc[endoderm.obs['time_points'].isin(['CS22-23']),['nhood_Sig_FC','early_late_time_points']] = ['Late','Late_CS22-23']

# Removal here

In [ ]:
macrophage

In [ ]:
objects = [macrophage,erythroid,endoderm]
objects_name = ['macrophage','erythroid','endoderm']

In [ ]:
cell_cycle_genes = pd.read_csv('/home/jovyan/Useful code and scripts/issac_cell_cycle_and_respiration/GO-0022402_cell_cycle_genes (1).csv', index_col=0, header=None)
cell_respr_genes = pd.read_csv('/home/jovyan/Useful code and scripts/issac_cell_cycle_and_respiration/GO-0045333_cell_respiration (1).csv', index_col=0, header=None)
cell_cycle_genes_list = list(cell_cycle_genes.index)
cell_respr_genes_list = list(cell_respr_genes.index)

In [ ]:
list_to_remove = []

for i in objects:
    remove = list(i.var.index[i.var_names.str.match("HB|MT-|RPS|RPL")])
    list_to_remove = list_to_remove + remove + cell_cycle_genes_list + cell_respr_genes_list
    list_to_remove = list(dict.fromkeys(list_to_remove))

In [ ]:
#list_to_remove

In [ ]:
n = 0
for x in objects:
    x = x[:,~x.var_names.isin(list_to_remove)]
    name = objects_name[n]
    globals()[name] = x
    n += 1

In [ ]:
objects = [macrophage,erythroid,endoderm]
for i in objects:
    print('Before pre-processing shape:')
    print(i.shape)
    #ud.describe_basic(i)
    sc.pp.filter_genes(i, min_cells=3)
    sc.pp.normalize_total(i)
    sc.pp.log1p(i)
    sc.pp.highly_variable_genes(i)
    print('After pre-processing shape:')
    #ud.describe_basic(i)
    print(i.shape)
    print('Loop complete')
    print('')

In [ ]:
deg_var = 'nhood_Sig_FC'
top_degs_to_plot = 20
data = macrophage
name = 'macrophage_early_vs_late_time_points_top_' + str(top_degs_to_plot) + '_only_sig_'
#name = 'macrophage_early_vs_late_time_points_top_' + str(top_degs_to_plot) + '_'

to_use = 'pvals'
#to_use = 'pvals_adj'

sc.tl.rank_genes_groups(data, deg_var, method='wilcoxon',n_genes=500)
sc.pl.rank_genes_groups(data, n_genes=50, sharey=False)
result = data.uns['rank_genes_groups']
groups = result['names'].dtype.names
    
Degs = pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', to_use,'logfoldchanges']}).head(100) # 100 to be conservative
Degs_name ="./Degs_unfiltered_211121.csv"
#Degs.to_csv(Degs_name)

Degs = pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', to_use,'logfoldchanges']})

# get dict of above
# filter out mito and ribo in list comprehension
# Then take head
#Degs.to_csv(Degs_name)

# side concatenation may occur  - may change order (known pandas error due to different lengths!)
# side concat in pandas good as long as length of rows the same 
Degs_minus_MT_RP = pd.DataFrame()
for i in data.obs[deg_var].unique():
    temp_deg = Degs.filter(like=i)
    temp_deg = temp_deg[temp_deg.iloc[:,0].str.match("MT-|RP")==False]
    temp_deg = temp_deg.reset_index(drop=True)
    Degs_minus_MT_RP = pd.concat([Degs_minus_MT_RP, temp_deg], axis=1)
Degs_minus_MT_RP_name ="./DEGS_mito_and_ribo_removed_211121.csv"

#Degs_minus_MT_RP.to_csv(Degs_minus_MT_RP_name)


# Note could just remove mito and ribo from data then re-calc as only thing which would change is pvals but minimal change



degs = Degs[:]
#degs = Degs_minus_MT_RP[:]

n = pd.melt(degs.loc[:, degs.columns.str.endswith("_n")])
p = pd.melt(degs.loc[:, degs.columns.str.endswith("_p")])
l = pd.melt(degs.loc[:, degs.columns.str.endswith("_l")])
n = n.replace(regex=r'_n', value='')
n = n.rename(columns={"variable": "cluster", "value": "gene"})
p = (p.drop(["variable"],axis = 1)).rename(columns={ "value": to_use})
l = (l.drop(["variable"],axis = 1)).rename(columns={ "value": "logfc"})
deg_table = pd.concat([n,p,l],axis=1)

if deg_table['logfc'].isnull().values.any() == True:
    print('\033[1m' +'logfc contains nans!')
    print()
    answer = input(f'Do you still want to sort the data by (logfc), if so input yes otherwise input no and will sort using {to_use} instead:')
    if answer == 'yes':
        deg_table_sub = deg_table.groupby('cluster').apply(lambda grp: grp.nlargest(top_degs_to_plot, 'logfc')).reset_index(drop=True)
        print()
    elif answer == 'no':
        deg_table_sub = deg_table.groupby('cluster').apply(lambda grp: grp.nsmallest(top_degs_to_plot, to_use)).reset_index(drop=True)
        print()
    else:
        raise Exception('Warning: Did not select yes or no exactly!')
else:
    deg_table_sub = deg_table.groupby('cluster').apply(lambda grp: grp.nlargest(top_degs_to_plot, 'logfc')).reset_index(drop=True)

#deg_table_sub = deg_table_sub.groupby('cluster').apply(lambda grp: grp.nsmallest(top_degs_to_plot, to_use)).reset_index(drop=True)


# significance 0.05
check = deg_table_sub[deg_table_sub[to_use] < 0.05][:]

# Checking if all clusters are captured!
if len(check['cluster'].unique()) < len(data.obs[deg_var].unique()):
    if check.empty == True:
        print(f'There are no significant genes in data using {to_use} <0.05! Using non-significant genes instead')
        print()
        to_use_deg_table_sub = deg_table_sub
        sig_answer = 'no'
        
    else:
        diff_genes = list(set(data.obs[deg_var].unique()).difference(check['cluster'].unique()))
        print('\033[1m' + f'Warning! There are clusters which have no significant genes after applying 0.05 cutoff using {to_use}!!')
        print()
        print(f'The clusters which have no significant genes  are: {diff_genes}')
        print()
        sig_answer = input('To continue with only significant genes input yes, otherwise to use no significant genes input no:')
        print()
        if sig_answer == 'no':
            to_use_deg_table_sub = deg_table_sub
        elif sig_answer == 'yes':
            print()
            print('Selected to plot only significant genes for each category')
            to_use_deg_table_sub = deg_table_sub[deg_table_sub[to_use] < 0.05]
            print()
        else:
            raise Exception("Error, did not input yes or no exactly!")
else:
    sig_answer = 'yes'

            
# Checking each cluster if it has top number of genes to plot
n_clusters = len(check['cluster'].unique())

if sig_answer != 'no':
    loop_n = 1
    for i in check['cluster'].unique():
        check_i = check[check['cluster'].isin([i])]
        if len(check_i) == top_degs_to_plot:
            if loop_n != n_clusters:
                pass
            else:
                to_use_deg_table_sub = deg_table_sub[deg_table_sub[to_use] < 0.05]
        else:
            answer = input(f'Warning! For category ({i}) there is not {top_degs_to_plot} genes which are <0.05 using {to_use}. To still plot only significant genes input yes, if not input no:')
            if answer == 'yes':
                print()
                print('Selected to plot only significant genes for each category')
                to_use_deg_table_sub = deg_table_sub[deg_table_sub[to_use] < 0.05]
                break
            elif answer == 'no':
                print()
                print(f'Selected to not restrict to just significant genes for each category, using top {top_degs_to_plot} for each category')
                to_use_deg_table_sub = deg_table_sub
                break
            else:
                raise Exception("Error, something went wrong - check the checking sig genes section")
        loop_n +=1
else:
    pass 
                

Genes = to_use_deg_table_sub.groupby('cluster')['gene'].apply(list).to_dict()
Genes = dict(sorted(Genes.items()))
Genes = {str(k):v for k,v in Genes.items()} # k= key, v= values
data_temp = data
#sc.pl.dotplot(data_temp, var_names = Genes, groupby=deg_var, standard_scale='var', color_map='Reds', show = True) # title=i save = 'degs.pdf'

# reset parmeters as these get changed frequently 
plt.rcdefaults()
sc.settings.set_figure_params(dpi=300)
sc.pl.dotplot(data_temp, var_names = Genes, groupby=deg_var, standard_scale='var', color_map='Reds', show = True ) # save=name+'dotplot.pdf'
sc.pl.matrixplot(data_temp, var_names = Genes, groupby=deg_var, standard_scale='var', show=True ) #save=name+'matrixplot.pdf'

In [ ]:
macrophage2 = macrophage[~macrophage.obs['nhood_Sig_FC'].str.contains('False')]

In [ ]:
sc.pp.scale(macrophage2, zero_center=True)

In [ ]:
dict1 = {
"iron_metabolism_and_scavenging":["ITGAM","TIMD4","MRC1","MERTK",'HMOX1',],
"fatty_acid_metabolism":['NR1H3','PLTP','APOC2','APOC3','APOE','ALB',], 
'MHCII_HIGH':["CLEC7A","HLA-DRA","HLA-DPA1",'HLA-DQB1','HLA-DRB1',]
}

sc.pl.matrixplot(macrophage2, var_names=dict1, groupby='time_points', use_raw=False, cmap='RdBu_r', vmax=1, vmin=-1, save = 'macrophage_functional_genesets_across_timepoints_matrixplot.pdf', colorbar_title='mean z-score',)